In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

stazioni = ["andria", "bari", "barletta_casardi", 
            "brindisi", "brindisi_sisri", "candela_ex_comes", 
            "carbonara", "casamassima", "cisternino", "foggia", 
            "francavilla_fontana", "galatina", "grottaglie", 
            "lecce", "lecce_cerrate", "maglie", "manfredonia_liceo", 
            "massafra", "molfetta", "monopoli", "san_pancrazio_salentino", 
            "san_severo", "taranto", "taranto_san_vito", "torchiarolo"]
anno = ["2019", "2020", "2021", "2022"]

In [10]:
def estrai_giorno(data): 
    return data[:10]

In [14]:
for i in range(0, len(stazioni)):
    dbs_max_min = []
    for j in range(0, 4):
        path = f"/Users/reginacastrovilli/paper/dati_arpa/{anno[j]}/dati_meteo_stazione_{stazioni[i]}_{anno[j]}.csv"
        file = pd.read_csv(path, sep=";")
        try:
            file = file.rename(columns={'temperatura_media': 'temperatura', 'umr_media': 'umr'})
        except KeyError:
            pass

        file['giorno'] = file['data'].apply(estrai_giorno)
        db_max_min = file.groupby('giorno').agg({'temperatura': ['max', 'min'],
                                                 'umr': ['max', 'min']}).reset_index()

        # Rinomina le colonne nel formato desiderato
        db_max_min.columns = ['giorno', 'temperatura_max', 'temperatura_min', 'umr_max', 'umr_min']

        Hs_max_t = []
        Hs_max_u = []
        Hs_min_t = []
        Hs_min_u = []

        for k in range(0, len(db_max_min)):
            giorno = db_max_min['giorno'].values[k]
            t_max = db_max_min['temperatura_max'].values[k]
            t_min = db_max_min['temperatura_min'].values[k]
            umr_max = db_max_min['umr_max'].values[k]
            umr_min = db_max_min['umr_min'].values[k]

            try:
                h_max_t = file[(file['giorno'] == giorno) & (file['temperatura'] == t_max)]['data'].values[0]
            except IndexError:
                h_max_t = np.nan

            try:
                h_max_u = file[(file['giorno'] == giorno) & (file['umr'] == umr_max)]['data'].values[0]
            except IndexError:
                h_max_u = np.nan

            try:
                h_min_t = file[(file['giorno'] == giorno) & (file['temperatura'] == t_min)]['data'].values[0]
            except IndexError:
                h_min_t = np.nan

            try:
                h_min_u = file[(file['giorno'] == giorno) & (file['umr'] == umr_min)]['data'].values[0]
            except IndexError:
                h_min_u = np.nan

            Hs_max_t.append(h_max_t)
            Hs_max_u.append(h_max_u)
            Hs_min_t.append(h_min_t)
            Hs_min_u.append(h_min_u)

        db_max_min['H_max_t'] = Hs_max_t
        db_max_min['H_max_u'] = Hs_max_u
        db_max_min['H_min_t'] = Hs_min_t
        db_max_min['H_min_u'] = Hs_min_u
        dbs_max_min.append(db_max_min)

    output_max_min = pd.concat(dbs_max_min)[['giorno', 'H_max_t', 'temperatura_max', 'H_min_t', 'temperatura_min',
                                             'H_max_u', 'umr_max', 'H_min_u', 'umr_min']]
    output_max_min.to_csv(f"/Users/reginacastrovilli/paper/Data_Max_Min/{stazioni[i]}.csv", sep=";", index=False)

In [13]:
dbs_max_min

[         giorno  temperatura_max  temperatura_min  umr_max  umr_min  \
 0    2019-01-01            13.60             8.39      NaN      NaN   
 1    2019-01-02            14.42             3.61      NaN      NaN   
 2    2019-01-03             8.22             3.86      NaN      NaN   
 3    2019-01-04             4.38             0.34      NaN      NaN   
 4    2019-01-05             8.68             0.39      NaN      NaN   
 ..          ...              ...              ...      ...      ...   
 360  2019-12-27            12.76             5.33    78.08    48.68   
 361  2019-12-28            10.45             6.08    65.41    41.68   
 362  2019-12-29            10.48             7.36    52.28    41.78   
 363  2019-12-30            10.44             6.69    57.40    40.18   
 364  2019-12-31            14.59             5.45    81.05    47.66   
 
                  H_max_t              H_max_u              H_min_t  \
 0    2019-01-01 15:00:00                  NaN  2019-01-01 23:0